# Configure the default docker network with systemd

Docker is a really good application for not destroying your system with high dependency applications. This is really useful, especially when you want to have a clean system (as much as possible).

Last day, we were studying databases in my engineering school, and we had to install `oracle-xe`. I really don't like installing these kinds of things on my system, because it's ugly, it does not work like every other things (no `pacman -S oracle-xe`) etc. I decided to use Docker to install `oracle-xe` in a particular environment, instead of directly install it on my system.

The point is that Docker use a bridge interface for running his containers. This bridge interface is pretty useful, except when... your `route -n` look like this :

    root@kripton ~ # route -n
    0.0.0.0         172.17.0.1      0.0.0.0         UG    202    0        0 eno1
    0.0.0.0         172.17.0.1      0.0.0.0         UG    0      0        0 docker0
    172.17.0.1      0.0.0.0         255.255.0.0     U     202    0        0 docker0

Because the Docker's route is more specific than your default route (the `eno1` route), when you want to connect on a computer in your LAN (let's say `172.17.0.24`), your system will route the packet through the `docker0` interface instead of the `eno1` interface. Conclusion ? You can't connect to this computer. Uuuuuh ;(

# How does it work ?

Docker is a container that manage your dependencies and your application runtime environment. In order to have his own network, it creates a virtual network interface also known as *bridged interface*. This interface allows you to connect to your application through a secured network, which is... pretty nice.

The bridge interface is created by docker at the daemon startup, and is globally managed by it if you do not configure your own interface. Because it's created by docker, it means that it has default values that might not fill with your system or network configuration.

My issue was here. The docker bridge interface default subnet was `172.17.0.0/16`, and my school network was on the subnet `172.17.0.0/16`. Because of this, I couldn't connect to my school VPN, and can not use Internet and Docker at the same time.

# Configure your bridge interface

If as me you didn't wrote anything in your `systemd` network management configuration, your `/etc/systemd/network` is empty. This folder contains all network interface and virtual interface configuration used by the `systemd-networkd` service in order to configure your network. In order to test your configuration, you can just restart the service :

```bash
systemctl restart systemd-networkd
```

## How to organise your `/etc/systemd/network` folder ?

Files are read in alphabetical order. As a lot other configuration folder like that, you should prefix all your files by a number in $[\![00, 99]\!]$.

Files have also 2 different extensions :

    *.netdev  for network device creation and setup
    *.network for network device configuration

Before going any further, you should read the `man systemd.network` and the `man systemd.netdev`. It's really useful to understand what means every sections and why we use them. Systemd's network configuration does not work like old configuration (through the `/etc/network/interfaces` file).

## Before doing anything, stop docker

You are going to make some huge modifications to the docker infrastructure. That might be harmful for docker, so just stop it before doing anything.

```bash
systemctl stop docker
```

## Create your docker's bridge network

The first thing to do is to create the bridge interface for docker. You can name this bridge as you want, but I like the `docker0` name (in case you want to have more than one instance of docker running).

In order to create it, you must create a `.netdev` file containing :

- the device name (`docker0` is an example)
- the device kind (see `man systemd.netdev` to see all available device kind)
- and if you want a device description.

```
$ cat /etc/systemd/network/20-bridge-docker0.netdev
[NetDev]
Name=docker0
Kind=bridge
Description=Docker bridge network
```

## Configure your main(s) interface(s)

*Before* configuring your docker bridge, setup your other interfaces. Here is my ethernet network configuration named `50-ethernet-dhcp.network` file that configure my `eno1` interface configuration :

```
$ cat /etc/systemd/network/50-ethernet-dhcp.network
[Match]
Name=eno1

[Network]
DHCP=yes
DNS=8.8.8.8
DNS=8.8.4.4
```

As you can see, my interface `eno1` is in DHCP mode. You can also configure it in static mode like this :

```
$ cat /etc/systemd/network/50-ethernet-static.network
[Match]
Name=eno1

[Network]
Address=192.168.0.15/24
Gateway=192.168.0.1
```

## Configure your bridge

Configuring your bridge interface is nearly the same as configuring your basic network interfaces. The main issue is that if you specify a gateway parameter in your configuration (obligatory for docker), it creates a default route for your system. Sometimes this default route overwrite your main interface default route (for me the `eno1`). The consequence of this is you can't access to internet anymore, which is quite annoying.

So we need a route specific route to access to our network, and we define it into the network configuration file.

```
$ cat /etc/systemd/network/75-bridge-static.network
[Match]
Name=docker0

[Network]
Address=10.10.254.254/24

[Route]
Gateway=10.10.254.1
Destination=10.10.254.0/24
```

## Tell docker which interface to use

The default interface docker use is `docker0`. But if for some reasons you want to rename this device, you must specificate to docker which device he has to use. I didn't find any other way by now, so here is what I did :

```
# cat /etc/systemd/system/multi-user.target.wants/docker.service 
[Unit]
Description=Docker Application Container Engine
Documentation=https://docs.docker.com
After=network.target docker.socket
Requires=docker.socket

[Service]
Type=notify
ExecStart=/usr/bin/docker daemon -b docker0 -H fd://
MountFlags=slave
LimitNOFILE=1048576
LimitNPROC=1048576
LimitCORE=infinity

[Install]
WantedBy=multi-user.target
```

The only thing I modificate in this file is the option `-d docker0` in the `Service.ExecStart` section. It specify docker to use the `docker0` bridge controller. And that's it.

## Restart your networkd and check if all is okay

After this, just reload `docker` and `systemd-networkd` and you're done !

```bash
systemctl restart systemd-networkd
```

At this point, your route *should* look like this :

```
# route -n
Kernel IP routing table
Destination     Gateway         Genmask         Flags Metric Ref    Use Iface
0.0.0.0         172.17.0.1      0.0.0.0         UG    202    0        0 eno1
10.10.254.0     0.0.0.0         255.255.255.0   U     0      0        0 docker0
10.10.254.0     10.10.254.1     255.255.255.0   UG    0      0        0 docker0
172.17.0.0      0.0.0.0         255.255.0.0     U     202    0        0 eno1
172.17.0.1      0.0.0.0         255.255.255.255 UH    1024   0        0 eno1
```

And your interfaces like this :

```
# ip a
1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default 
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: eno1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc fq_codel state UP group default qlen 1000
    link/ether ec:ff:bb:cc:00:22 brd ff:ff:ff:ff:ff:ff
    inet 172.17.7.204/16 brd 172.17.255.255 scope global dynamic eno1
       valid_lft 169262sec preferred_lft 169262sec
    inet 172.17.9.40/16 brd 172.17.255.255 scope global secondary dynamic eno1
       valid_lft 170251sec preferred_lft 170251sec
    inet 172.17.8.133/16 brd 172.17.255.255 scope global secondary eno1
       valid_lft forever preferred_lft forever
    inet6 fe80::d569:a090:c38d:5a6/64 scope link 
       valid_lft forever preferred_lft forever
31: docker0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default 
    link/ether 02:42:f9:ce:3c:c8 brd ff:ff:ff:ff:ff:ff
    inet 10.10.254.254/24 brd 10.10.254.255 scope global docker0
       valid_lft forever preferred_lft forever
    inet6 fe80::d252:9a4:c734:a541/64 scope link 
       valid_lft forever preferred_lft forever
```

*Little point* : sometimes, the interface `docker0` might keep some traces of the old configuration (setup by docker). You might have something like this :

```
# ip a
.../...
31: docker0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default 
    link/ether 02:42:f9:ce:3c:c8 brd ff:ff:ff:ff:ff:ff
    inet 10.10.254.254/24 brd 10.10.254.255 scope global docker0
       valid_lft forever preferred_lft forever
    inet 172.17.0.1/24 brd 172.17.0.255 scope global docker0
       valid_lft forever preferred_lft forever
    inet6 fe80::d252:9a4:c734:a541/64 scope link 
       valid_lft forever preferred_lft forever
```

In order to delete this, just tell ip to delete the address on your bridge interface

```bash
ip addr del 127.17.0.1/24 dev docker0
```

You're done with this part if :

- your bridged interface is configured on the expected sub-network (for example `10.10.254.254/24`)
- you have a route that points to this sub-network as this :

```
    Destination     Gateway         Genmask         Flags Metric Ref    Use Iface
    10.10.254.0     10.10.254.1     255.255.255.0   UG    0      0        0 docker0
```
    
- you do NOT have a default route using your bridge network
- your default route use your main interface like this

```
    Destination     Gateway         Genmask         Flags Metric Ref    Use Iface
    0.0.0.0         172.17.0.1      0.0.0.0         UG    202    0        0 eno1
```

## Time to restart docker and enjoy !

You can now restart docker.

```bash
systemctl restart docker
```

# Any issues ?

Please tell me !